In [93]:
import findspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import col, desc

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [94]:
sc = spark.sparkContext

Los datos adjuntos a esta lección forman parte de la base de datos [NeurIPS 2020] Data Science for COVID-19 (DS4C) disponible en Kaggle. Estos datos hacen referencia a los casos de contagio de covid-19 en Corea del Sur.

El archivo csv Case contiene los casos reportados y el archivo csv PatientInfo contiene la información de los pacientes.

A partir del archivo csv Case, determine las tres ciudades con más casos confirmados de la enfermedad. La salida debe contener tres columnas: provincia, ciudad y casos confirmados. El resultado debe contener exactamente los tres nombre de ciudades con más casos confirmados ya que no se admiten otros valores.

In [95]:
df_casos = spark.read.option('header', 'true').option('inferSchema', 'true').csv('./data/Case.csv')

In [96]:
df_casos.printSchema()

root
 |--  case_id: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: boolean (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [97]:
df_casos = df_casos.withColumnRenamed(' case_id', 'case_id')
df_casos.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: boolean (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [98]:
df_casos.show(5)

+-------+--------+------------+-----+--------------------+---------+---------+----------+
|case_id|province|        city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|  Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
|1000002|   Seoul|   Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
|1000003|   Seoul|     Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
|1000004|   Seoul|Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
|1000005|   Seoul|   Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



In [99]:
df_casos.filter((col('city') != '-') & (col('city') != 'from other city')).orderBy(desc('confirmed')).select('province', 'city', 'confirmed').show(3)

+--------+------------+---------+
|province|        city|confirmed|
+--------+------------+---------+
|   Daegu|      Nam-gu|     4511|
|   Daegu|Dalseong-gun|      196|
|   Seoul|  Yongsan-gu|      139|
+--------+------------+---------+
only showing top 3 rows



Cree un dataframe a partir del archivo csv PatientInfo. Asegúrese de que su dataframe no contenga pacientes duplicados.

    1.- ¿Cuántos pacientes tienen informado por quién se contagiaron(columna infected_by)? Obtenga solo los pacientes que tengan informado por quién se contagiaron.
    2.- A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.
    3.- Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.


In [100]:
df_paciente = spark.read.option('header', 'true').option('inferSchema', 'true').csv('./data/PatientInfo.csv').dropDuplicates(['patient_id'])

In [101]:
df_paciente.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

In [102]:
# Ahora comprobamos que no hay duplicados comparando el número total de filas con el número total de distinc values en la columna 'patient_id'

df_paciente.select(countDistinct('patient_id')).first()['count(DISTINCT patient_id)'] == df_paciente.select(count('patient_id')).first()['count(patient_id)']

True

In [103]:
# 1

df_infected_by = df_paciente.na.drop(subset=['infected_by'])
df_infected_by.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   2020-02-19|         null|released|
|1000000005|female|20s|  Korea|   Seoul|Seongbuk-gu|contact with patient| 1000000002|             2|              null|    2020-01-31|   2020-02-24|         null|released|
|1000000006|female|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 1000000003|            43|              null|    2020-01-31|   202

In [104]:
number = df_infected_by.select(count('patient_id')).first()['count(patient_id)']
print(f'{number} pacientes han informado por quién se contagiaron')

1346 pacientes han informado por quién se contagiaron


In [105]:
# 2

df_infected_by_fem = df_infected_by.filter(col('sex')=='female').drop('released_date', 'deceased_date')
df_infected_by_fem.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+--------+
|1000000005|female|20s|  Korea|   Seoul|Seongbuk-gu|contact with patient| 1000000002|             2|              null|    2020-01-31|released|
|1000000006|female|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 1000000003|            43|              null|    2020-01-31|released|
|1000000010|female|60s|  Korea|   Seoul|Seongbuk-gu|contact with patient| 1000000003|             6|              null|    2020-02-05|released|
|1000000014|female|60s|  Korea|   Seoul|  Jongno-gu|contact with patient| 1000000013|            27|        2020-02-06|    2020-02-16|re

In [106]:
# 3

df_part2.coalesce(2).write.partitionBy('province').mode('overwrite').parquet('./data/korea_covid')